In [1]:
from utils import *

In [2]:
data_dir = '/datadrive/data/'
graph = dill.load(open(data_dir + 'graph.pk', 'rb'))

In [3]:
train_range = {t: True for t in graph.times if t != None and t <= 2015}
valid_range = {t: True for t in graph.times if t != None and (t > 2015) & (t < 2018)}
test_range  = {t: True for t in graph.times if t != None and t >= 2018}

In [4]:
class Matcher(nn.Module):
    def __init__(self, n_hid, dropout = 0.5):
        super(Matcher, self).__init__()
        self.left_linear    = nn.Linear(n_hid,  n_hid)
        self.right_linear   = nn.Linear(n_hid,  n_hid)
        self.drop     = nn.Dropout(dropout)
        self.mem      = None
    def forward(self, x, y, test = False):
        if test:
            if self.mem != None:
                tx = self.mem
            else:
                tx = self.left_linear(x)
                self.mem = tx
        else:
            tx = self.drop(self.left_linear(x))
        ty = self.drop(self.right_linear(y))
        return torch.log_softmax(torch.mm(ty, tx.T), dim=-1)

In [5]:
def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
    return 0.

def ndcg_at_k(r, k):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k) / dcg_max

def mean_reciprocal_rank(rs):
    rs = (np.asarray(r).nonzero()[0] for r in rs)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in rs])

In [6]:
r = [1,1,1, 0, 0, 1, 1, 0]
print(mean_reciprocal_rank(r), ndcg_at_k(r, 10))

0.625 0.9473224867337506


In [7]:
batch_size = 512
batch_num  = 128
epoch_num  = 100
samp_num   = 7

device = torch.device("cuda:1")
train_feature, train_time, train_edge_list = \
        graph.sample_subgraph(time_range=train_range, sampled_depth = 4, sampled_number = 64)
node_feature, node_type, edge_time, edge_index, edge_type, node_dict, edge_dict  = \
        to_torch(train_feature, train_time, train_edge_list, device)

In [41]:
def sample_subgraph(graph, time_range, sampled_depth = 2, sampled_number = 8, inp = False):
    '''
    Sample Sub-Graph based on a subset of paper (events) and their happening time
    '''
    def add_budget(te, target_id, target_time, layer_data, budget, maxnum = sampled_depth * sampled_number):
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                if relation_type == 'self':
                    continue
                adl = tes[relation_type][target_id]
                if len(adl) < maxnum:
                    sampled_ids = list(adl.keys())
                else:
                    sampled_ids = np.random.choice(list(adl.keys()), maxnum, replace = False)
                for source_id in sampled_ids:
                    source_time = adl[source_id]
                    if source_time == None:
                        source_time = target_time
                    k = encode(source_id, source_time)
                    if source_time not in time_range or k in layer_data[source_type]:
                        continue
                    budget[source_type][k] += 1. / len(sampled_ids)
    def decode(s):
        idx = s.find('-')
        return np.array([s[:idx], s[idx+1:]], dtype=float)
    def encode(i, t):
        return '%s-%s' % (i, t)

    layer_data  = defaultdict( #target_type
                        lambda: {} # {target_id + time}
                    )
    budget     = defaultdict( #source_type
                                lambda: defaultdict(  #source_id + source_time
                                    lambda: 0. #sampled_score
                            ))
    new_layer_adj  = defaultdict( #target_type
                                    lambda: defaultdict(  #source_type
                                        lambda: defaultdict(  #relation_type
                                            lambda: [] #[target_id, source_id]
                                )))
    
    if inp == None:
        rand_paper_ids  = np.random.choice(range(len(graph.node_feature['paper'])), sampled_number * 2, replace = False)
        rand_paper_time = np.array(list(graph.node_feature['paper'].loc[rand_paper_ids, 'time']))
        for _id, _time in zip(rand_paper_ids, rand_paper_time):
            if _time not in time_range:
                continue
            layer_data['paper'][encode(_id, _time)] = len(layer_data['paper'])
            add_budget(graph.edge_list['paper'], _id, _time, layer_data, budget)
    else:
        '''
        budget: {_type: [[_id, _time]]}
        '''
        for _type in inp:
            for _id, _time in inp[_type]:
                layer_data[_type][encode(_id, _time)] = len(layer_data[_type])
                add_budget(graph.edge_list[_type], _id, _time, layer_data, budget)
            
    for layer in range(sampled_depth):
        for source_type in graph.get_types():
            keys  = np.array(list(budget[source_type].keys()))
            if sampled_number > len(keys):
                sampled_ids = np.arange(len(keys))
            else:
                score = np.array(list(budget[source_type].values()))
                score = score / np.sum(score)
                sampled_ids = np.random.choice(len(score), sampled_number, p = score, replace = False) 
            sampled_ids = keys[sampled_ids]
            for k in sampled_ids:
                source_id, source_time = decode(k)
                layer_data[source_type][k] = len(layer_data[source_type])
                add_budget(graph.edge_list[source_type], int(source_id), int(source_time), layer_data, budget)
            for k in sampled_ids:
                budget[source_type].pop(k)
                
    print('Finish Propagate Nodes')            
    
    feature = {}
    times   = {}
    for target_type in layer_data:
        idxs  = np.array([decode(key) for key in layer_data[target_type]])
        feature[target_type] = list(graph.node_feature[target_type].loc[idxs[:,0], 'w2v'])
        times[target_type]   = idxs[:,1]
    edge_list = defaultdict( #target_type
                        lambda: defaultdict(  #source_type
                            lambda: defaultdict(  #relation_type
                                lambda: [] # [target_id, source_id] 
                                    )))
    for target_type in layer_data:
        for target_id in layer_data[target_type]:
            target_ser = layer_data[target_type][target_id]
            edge_list[target_type][target_type]['self'] += [[target_ser, target_ser]]
    for target_type in graph.edge_list:
        te = graph.edge_list[target_type]
        for source_type in te:
            tes = te[source_type]
            for relation_type in tes:
                tesr = tes[relation_type]
                for target_id in layer_data[target_type]:
                    target_ser = layer_data[target_type][target_id]
                    for source_id in layer_data[source_type]:
                        source_ser = layer_data[source_type][source_id]
                        if decode(source_id)[0] in tesr[decode(target_id)[0]]:
                            edge_list[target_type][source_type][relation_type] += [[target_ser, source_ser]]
    return feature, times, edge_list

In [42]:
inp = {'paper': np.array([np.arange(10), np.ones(10) * 2018]).T}
feature, times, edge_list = sample_subgraph(graph, train_range, inp = inp)

Finish Propagate Nodes


In [44]:
times

{'paper': array([2018., 2018., 2018., 2018., 2018., 2018., 2018., 2018., 2018.,
        2018., 2014., 2014., 2014., 2011., 2014., 2015., 2013., 2015.,
        1991., 1987., 2005., 2015., 1983., 2011., 2007., 2013.]),
 'venue': array([1993., 2014., 2014., 2004., 1991., 2013., 2015., 2011., 1965.,
        2015., 2010., 1998., 2010., 2011., 2011., 2005.]),
 'field': array([2010., 2015., 2004., 1993., 2014., 2010., 2004., 2011., 2004.,
        1998., 2011., 2010., 1998., 2015., 2014., 2005.]),
 'author': array([2011., 2013., 2015., 2015., 2004., 2013., 1993., 2015., 1993.,
        2015., 2015., 2010., 2010., 2011., 2011., 2015.]),
 'affiliation': array([2015., 1993., 2015., 2015., 2015., 2011., 2004., 2013., 2010.,
        2013., 2011., 2010., 2015., 2013., 2015., 2015.])}

In [18]:
'''
Paper-Venue
'''
field_dict = dill.load(open(data_dir + 'venue_dict.pk', 'rb'))
ids = np.array([graph.node_forward['venue'][k] for k in field_dict if field_dict[k][0] == 'L1'])

paper_ser = {}

train_pairs = defaultdict(lambda: [])
valid_pairs = defaultdict(lambda: [])
test_pairs  = defaultdict(lambda: [])

train_sers = []
valid_sers = []
test_sers  = []

for f_ser, f_id in enumerate(ids):
    for p_id in graph.edge_list['venue']['paper']['PV_in'][f_id]:
        _time = graph.edge_list['venue']['paper']['PV_in'][f_id][p_id]
        if _time in train_range:
            train_pairs[p_id] += [f_id]
            train_sers  += [p_id]
        elif _time in valid_range:
            valid_pairs[p_id] += [f_id]
            valid_sers  += [p_id]
        else:
            test_pairs[p_id] += [f_id]
            test_sers  += [p_id]

KeyboardInterrupt: 

In [ ]:
matcher = Matcher(400).to(device)
optimizer = torch.optim.Adam(matcher.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 2000, eta_min=1e-5)

In [ ]:
stats = []
train_step = 0
best_val   = 0
res = []
criterion = nn.KLDivLoss(reduction='batchmean')
for epoch in np.arange(epoch_num)+1:
    '''
        Train
    '''
    matcher.train()
    train_losses = []
    ylabel = torch.zeros(batch_size, len(field_vecs)).to(device)
    for batch in np.arange(batch_num)+1:
        train_xids = train_sers[np.random.choice(len(train_sers), batch_size)]
        for yl, _id in zip(ylabel, train_xids):
            yl[np.array(train_pairs[_id])] = 1
        train_ylabel = ylabel / ylabel.sum(dim=-1).view(-1, 1)
        train_pvecs = torch.FloatTensor(paper_vecs[train_xids]).to(device)
        pred = matcher.forward(field_vecs, train_pvecs)
        loss = criterion(pred, train_ylabel)
        
        optimizer.zero_grad() 
        loss.backward()
        optimizer.step()
        train_losses += [loss.cpu().detach().tolist()]
        train_step += 1
        scheduler.step(train_step)
    '''
        Valid
    '''
    matcher.eval()
    valid_xids = valid_sers[np.random.choice(len(valid_sers), batch_size)]
    ylabel = torch.zeros(batch_size, len(field_vecs)).to(device)
    for yl, _id in zip(ylabel, valid_xids):
        yl[np.array(valid_pairs[_id])] = 1
    valid_ylabel = ylabel / ylabel.sum(dim=-1).view(-1, 1)
    valid_pvecs = torch.FloatTensor(paper_vecs[valid_xids]).to(device)
    pred = matcher.forward(field_vecs, valid_pvecs)
    loss = criterion(pred, valid_ylabel)
    valid_res = []
    
    for ai, bi in zip(ylabel, pred.argsort(descending = True)):
        valid_res += [ai[bi].tolist()]
        
    '''
        Test
    '''
    test_xids = test_sers[np.random.choice(len(test_sers), batch_size)]
    ylabel = torch.zeros(batch_size, len(field_vecs)).to(device)
    for yl, _id in zip(ylabel, valid_xids):
        yl[np.array(valid_pairs[_id])] = 1
    test_pvecs = torch.FloatTensor(paper_vecs[test_xids]).to(device)
    pred = matcher.forward(field_vecs, test_pvecs)
    test_res = []
    
    for ai, bi in zip(ylabel, pred.argsort(descending = True)):
        test_res += [ai[bi].tolist()]
    if np.average([ndcg_at_k(resi, 100) for resi in valid_res]) > best_val:
        best_val = np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res])
        res = [np.average([ndcg_at_k(resi, len(resi)) for resi in test_res]),\
          np.average([ndcg_at_k(resi, 100) for resi in test_res]), np.average([mean_reciprocal_rank(resi) for resi in test_res])]
    print(("Epoch: %d  LR: %.4f Train Loss: %.2f  Valid Loss: %.2f  Valid NDCG: %.4f  Test NDCG: %.4f  Test NDCG@100: %.4f  Test MRR: %.4f") % \
          (epoch, optimizer.param_groups[0]['lr'], np.average(train_losses), loss.cpu().detach().tolist(),\
          np.average([ndcg_at_k(resi, len(resi)) for resi in valid_res]), np.average([ndcg_at_k(resi, len(resi)) for resi in test_res]),\
          np.average([ndcg_at_k(resi, 100) for resi in test_res]), np.average([mean_reciprocal_rank(resi) for resi in test_res])))